# 将决策问题转化为优化模型

在第一个视频中，我们讨论了数学优化所需的几个关键概念：
- 参数 
- 决策变量
- 约束条件
- 目标函数

在这第一个建模示例中，我们将看到如何使用这些概念将决策问题表述为优化模型，并使用`gurobipy`编码实现这个公式化。有关Python API中所有命令的更多信息，请查看我们的[文档](https://www.gurobi.com/documentation/10.0/refman/py_python_api_details.html)。

## 决策问题
我们生产部件。拥有一组生产设施，生产装有部件的箱子。还有一组分销地点，将这些部件分销销售。每个分销中心都有预测需求，而每个生产设施在此期间能够生产的部件数量有最小值和最大值。我们需要确保每个分销设施从生产处接收足够的部件以满足需求，并且我们希望以最小成本实现这一点。最小生产量是生产设施最大值的75%。

## 集合和定义模型
我们的集合是：
- $P = \{\texttt{'Baltimore', 'Cleveland', 'Little Rock', 'Birmingham', 'Charleston'}\} \quad\quad\quad\quad\quad\quad\quad\space\space \texttt{production}$
- $D = \{\texttt{'Columbia', 'Indianapolis', 'Lexington', 'Nashville', 'Richmond', 'St. Louis'}\} \quad\quad\quad \texttt{distribution}$

为了索引每个集合，我们将使用每个集合的小写字母。用于集合和索引的字母由您决定。通常，大写字母用于集合，相应的小写字母将作为索引。单个字母主要用于简洁。

In [2]:
# %pip install gurobipy

# 导入包
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# 分别设置集合P和D
# 当我们编写集合时，可以使用更具描述性的名称
production = ['Baltimore','Cleveland','Little Rock','Birmingham','Charleston']
distribution = ['Columbia','Indianapolis','Lexington','Nashville','Richmond','St. Louis']

# 为决策问题定义一个gurobipy模型
m = gp.Model('widgets')

## 参数

数学优化问题的参数是在模型中被视为常量的值，与决策变量相关联。对于这个决策问题，这些值是每个生产设施的限制、每个分销中心的需求，以及生产和分销地点之间的成本。

- $m_p$ 是位置 $p$ 的最大生产量，$\forall p \in P \quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\space\space \texttt{max}\_\texttt{prod[p]}$
- $n_d$ 是分销中心 $d$ 的客户数量，$\forall d \in D \quad\quad\quad\quad\quad\quad\quad\quad \texttt{n}\_\texttt{demand[d]}$
- $c_{p,d}$ 是在位置 $p$ 和位置 $d$ 之间运送部件的成本，$\forall p \in P, d \in D \quad\quad\quad \texttt{cost[p,d]}$

In [3]:
# 使用.squeeze("columns")将成本转换为系列
path = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/Modeling_Session_1/'
transp_cost = pd.read_csv(path + 'cost.csv', index_col=[0,1]).squeeze("columns")
# transp_cost = pd.read_csv('cost.csv', index_col=[0,1]).squeeze("columns")
# 透视表格以便更容易查看成本
transp_cost.reset_index().pivot(index='production', columns='distribution', values='cost')

distribution,Columbia,Indianapolis,Lexington,Nashville,Richmond,St. Louis
production,,,,,,
Baltimore,4.50,5.09,4.33,5.96,1.96,7.30
Birmingham,3.33,4.33,3.38,1.53,5.95,4.01
Charleston,3.02,2.61,1.61,4.44,2.36,4.60
Cleveland,2.43,2.37,2.54,4.13,3.20,4.88
Little Rock,6.42,4.83,3.39,4.40,7.44,2.92


In [4]:
max_prod = pd.Series([180,200,140,80,180], index = production, name = "max_production")
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name = "demand") 
max_prod.to_frame()
#n_demand.to_frame()

,max_production
Baltimore,180
Cleveland,200
Little Rock,140
Birmingham,80
Charleston,180


我们还有每个生产设施需要达到其最大产能75%的要求。我们将在公式化中用 $a$ 表示这个值，用"frac"表示所需的最大产能比例。最初我们设定 $a = 0.75$。

In [5]:
frac = 0.75

## 决策变量
这是优化求解器确定的内容，即您可以控制的行动。提醒一下，它们主要有三种形式：
- `连续型`：产品价格
- `整数型`：用于活动的食品卡车数量
- `二元型`：投资组合中是否包含某支股票的是/否决策

决策变量（和参数）使用我们为问题定义的集合元素进行索引。在这个例子中，让我们从一组生产我们部件的城市开始，我们在公式中称之为集合 $P$，但在代码中可以定义为'production'。以及一组分销部件的城市 $D$，类似地定义为'distribution'。这里的决策是确定从每个生产设施到每个分销地点发送的箱子数量。

令 $x_{p,d}$ 表示在设施 $p$ 生产并运送到地点 $d$ 的部件数量。

### 在gurobipy中添加变量
`gurobipy` 主要通过两个（相似的）命令让您添加决策变量：
- [addVar()](https://www.gurobi.com/documentation/10.0/refman/py_model_addvar.html) 添加单个变量
- [addVars()](https://www.gurobi.com/documentation/10.0/refman/py_model_addvar.html) 通过集合/索引添加一组变量

使用 `addVars` 时，您必须提供要添加的变量的索引，对我们来说，这些是生产和分销地点。我们可以使用其他参数，稍后将介绍其中几个。

### 我们的决策变量
在编写代码时，通常有几种方法可以达到同样的目的。下面我们可以看到创建决策变量的三种不同方式。

In [6]:
# 遍历每个p和d组合来创建决策变量
m = gp.Model('widgets')


In [7]:
# 为索引提供每个集合
m = gp.Model('widgets')


In [8]:
# 运输成本的索引包含生产和分销地点的每种组合
m = gp.Model('widgets')


命令 `m.update()` 更新模型以包含所做的任何更改，如添加变量。它不需要在每个单元格中运行，但如果您在单元格的输出中看到 *Awaiting Model Update*，那么这应该可以防止这种情况发生。

## 约束条件
我们在本示例开始时概述了生产和需求约束；现在我们将它们公式化并编码。请注意，向模型添加约束（和/或决策变量）的顺序并不重要。

### 在gurobipy中添加约束
向模型添加约束与添加变量类似：
- [addConstr()](https://www.gurobi.com/documentation/10.0/refman/py_model_addconstr.html) 添加单个约束
- [addConstrs()](https://www.gurobi.com/documentation/10.0/refman/py_model_addconstrs.htmll) 使用Python `generator` 表达式添加一组约束
 
### 我们的约束
首先，我们将为每个分销地点制定需求约束，并将它们添加到模型中。

\begin{align*} 
\sum_{p}x_{p,d} \ge n_d, \quad \forall d \in D \quad\quad \texttt{meet}\_\texttt{demand[d]}\\ 
\end{align*}

这将是我们第一次使用 [gp.quicksum()](https://www.gurobi.com/documentation/10.0/refman/py_quicksum.html)。在gurobipy中有其他方法可以对表达式求和，虽然这种方法在编码上不是最简洁的，但它很容易与公式中的求和进行比较，以了解它的工作原理。

接下来，我们有每个生产设施可以制造的最大部件数量。我们还要求每个设施必须至少达到其最大产能的75%。

$$
\begin{align*} 
\sum_{d}x_{p,d} &\le m_p, &\forall p \in P \quad\quad &\texttt{can}\_\texttt{produce[p]}\\ 
\sum_{d}x_{p,d} &\ge a*m_p,&\forall p \in P \quad\quad &\texttt{must}\_\texttt{produce[p]}\\ 
\end{align*}
$$


## 目标函数
我们被要求**减少**运输成本，我们将用这一点来确定我们的目标函数，即最小化从生产地到分销地运送部件的总成本。

### 在gurobipy中设置目标
这是通过 [setObjective()](https://www.gurobi.com/documentation/10.0/refman/py_model_setobjective.html) 完成的。第二个参数（在这种情况下是 `GRB.MINIMIZE`）被称为模型的*sense*。对于最大化问题，我们将使用 `GRB.MAXIMIZE`。

### 我们的目标函数
\begin{align*} 
{\rm minimize} \space \sum_{p,d}c_{p,d}x_{p,d}, \quad \forall p \in P, d \in D\\ 
\end{align*}

## 查找、提取和分析解决方案
在运行优化之前，写入一个 `lp` 文件是一个好主意。这是一个文本文件，它像我们在*公式化*中看到的那样打印出变量、约束和对象，只是没有求和符号，并使用我们指定的名称。

In [ ]:
m.write('widget_shipment.lp')

### 运行优化

In [ ]:
m.optimize()

### 提取解决方案
从gurobipy中获取决策变量值的方法有很多。

In [ ]:
x_values = pd.Series(m.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol = pd.concat([transp_cost, x_values], axis=1)
#sol 
sol[sol.shipment > 0]

这里还有几种其他获取解决方案的方法。

In [ ]:
# 你可以获取所有决策变量的名称和值：
all_vars = {v.varName: v.x for v in m.getVars()} 
all_vars

或者，您可以只迭代特定变量，并仅返回您感兴趣的值。记住，x在python中是一个字典。所以，就像迭代任何字典一样迭代它

In [ ]:
xvals = {k: v.x for k,v in x.items() if v.x > 0} 
xvals 

### 解决方案分析
虽然确定部件的最佳运输是我们的目标，但我们可能想要更深入地研究解决方案。例如，我们可以按设施汇总总生产量，看看哪些地点（如果有的话）没有生产其最大产能的部件，以及哪些（如果有的话）生产设施处于其生产下限。

In [ ]:
# 按生产设施汇总运输量
ship_out = sol.groupby('production')['shipment'].sum()
pd.DataFrame({'Remaining':max_prod - ship_out, 'Utilization':ship_out/max_prod})

在数学优化中，当不等式约束的左侧和右侧相等时，我们说约束是`绑定的`。当这种情况*没有发生*时，该约束中就存在`松弛`或`剩余`。我们可以通过调用约束的 `Slack` 属性来获取此值。

In [ ]:
pd.DataFrame({'Remaining':[can_produce[p].Slack for p in production], 
              'Utilization':[1-can_produce[p].Slack/max_prod[p] for p in production]}, 
             index = production)

# 使用二元变量
正如我们在第一节和本笔记本顶部所描述的，二元变量用于在数学优化中选择替代方案。它们可以解释为是/否决策或开/关开关。

在原始问题中，Birmingham的产量比其他设施低得多。假设我们有选项将该设施的最大产能增加25或50个部件，但选择其中一个选项需要支付50美元和75美元的成本，并且我们最多只能选择一个。我们将为每个选项使用一个名为$xprod$的二元决策变量。

令 $xprod_0 = 1$ 如果我们选择第一个选项并将生产能力增加25，否则为 $0$。
令 $xprod_1 = 1$ 如果我们选择第二个选项并将生产能力增加50，否则为 $0$。

虽然使用单个小写字母作为决策变量相当常见，但这不是必须的，您将经常看到如上定义的变量（更具描述性）。我们将制定一个新模型，其中包含与之前相同的决策变量和需求约束。

In [ ]:
# 我们使用m2作为第二个模型
# 除了新模型名称外，这些部分与上面相同
m2 = gp.Model('widgets2')
x = m2.addVars(production, distribution, obj = transp_cost, name = 'prod_ship')
meet_demand = m2.addConstrs((gp.quicksum(x[p,d] for p in production) >= n_demand[d] for d in distribution), name = 'meet_demand')

在上面的单元格中，我们确实使用了 `addVars()` 函数的一个新参数：`obj`。这将设置添加的决策变量在目标函数中的系数，相当于我们之前通过将每个生产和分销地点之间的运输成本附加到适当的决策变量所做的操作。

接下来，我们将为除Birmingham以外的每个生产设施添加与之前相同的生产限制约束。公式基本相同，只是约束适用的集合不同。
$$
\begin{align*} 
\sum_{d}x_{p,d} &\le m_p, &\forall p \in P -\{\texttt{Birmingham}\} \\ 
\sum_{d}x_{p,d} &\ge a*m_p,&\forall p \in P -\{\texttt{Birmingham}\} \\ 
\end{align*}
$$

在gurobipy中，这是通过在 `generator` 表达式中添加条件来完成的。

现在，添加新的二元变量。

让我们分解上面单元格中的每个参数 —— 那里有一些新东西。
1. `range(n)` 用于添加 $n$ 个决策变量。在这种情况下，我们添加2个变量。
2. 我们需要使用 `vtype` 将其声明为二元变量。
3. 我们再次使用 `obj` 功能立即设置这些变量的目标函数系数。

目标和新的二元变量在公式中如下所示：

\begin{align*} 
{\rm minimize} \space &\sum_{p,d}c_{p,d}x_{p,d} + 50*xprod_0 + 75*xprod_1, \quad &\forall p \in P, d \in D\\ 
&xprod_i \in \{0,1\}, &{\rm for} \space i \in \{0,1\}
\end{align*}

接下来我们有特定于Birmingham设施的生产约束。

$$
\begin{align*} 
\sum_{d}x_{p,d} &\le m_p + 25*xprod_0 + 50*xprod_1, & p = \texttt{Birmingham} \\ 
\sum_{d}x_{p,d} &\ge a*(m_p+ 25*xprod_0 + 50*xprod_1),& p = \texttt{Birmingham} \\ 
\end{align*}
$$

上面提到，我们最多可以选择一个扩展选项，这意味着我们不能允许 $xprod_0$ 和 $xprod_1$ 同时等于1。为了对此进行建模，我们添加一个约束，将这两个二元变量的总和限制为最多1。

$$
\begin{align*}
\sum_{i}xprod_i \le 1
\end{align*}
$$
gurobipy中相应的约束：

现在我们可以运行这个优化模型，看看这个潜在的扩展是否会帮助我们降低总体成本。

In [ ]:
m2.optimize()

In [ ]:
obj1 = m.getObjective()
obj2 = m2.getObjective()
print(f"原始模型的总成本为 {round(obj1.getValue(),2)}")
print(f"新公式的总成本为 {round(obj2.getValue(),2)}")

目标函数值的变化告诉我们什么？

让我们看看二元变量的值。

In [ ]:
pd.Series(m2.getAttr('X', xprod))

模型选择了第一个扩展选项，因为 $xprod_0 = 1$，即将Birmingham的产能增加25个部件。我们可以看到其余的解决方案，其中将包括Birmingham的产能增加。

In [ ]:
x2_values = pd.Series(m2.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol2 = pd.concat([transp_cost, x2_values], axis=1)
sol2[sol2.shipment > 0]

### 作业！（并不是真的，但值得研究的东西）
分析两个模型之间最优解的变化。你会注意到一些奇怪的事情。
- 有什么奇怪的地方？
- 你认为为什么会发生这种情况？
- 从公式化角度和商业角度，你会如何解决它？